In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares_noOutputs
from mGovernment import BalancedBudget

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.


# Government module

The government module consists of two parts: First, they produce government services by combining inputs from domestic and foreign domestic sectors. This part is described (almost) as a typical production with a CES nesting. Second, they manage the budget.

In this version, we assume that the level of government services remain constant over time and that the budget is balanced every year by adjusting the tax on labor income. As the 
tax on labor income is defined in the household module, this does not work as a partial equilibrium model.

Causality in balanced budget part:
* The tax rate on labor income in the baseline year is *exogenous* when calibrating. Because of incomplete information about assets, debt - and because we only have a couple of tax instruments included in the model, we model a "balanced budget" by endogenizing a "jTerm" variable and let this be the target "balance" for each year.
* The total tax revenue collected from the government sector is targeted like in the production sectors by adjusting all demand taxes up/down proportionally until we reach the correct level.
* The demand for inputs from production sectors is matched by adjusting share parameters in CES nests.

*Load data:*

In [2]:
t0 = 2019 # set baseline year
name = 'vA' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}')) # Load 
ws = db.ws # run everything from the same ws

## 1. Nesting tree and value shares

Nesting tree:

In [3]:
cesNest = nestingTree.Tree('G', tree = db('nestG').to_list(), f = 'CES')
nest = nestingTree.AggTree(name = 'G', trees = {t.name: t for t in [cesNest]}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')})

Value shares:

In [4]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(db)

We are particularly interested in the $\mu$ variable and value of intermediate goods (to get better initial values):

In [5]:
db['mu'] = db_vs('mu').xs(t0) 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

## 2. Partial Equilibirum Model

Only keep household sectors, extrapolate data for initial values:

Initialize module from nesting tree and add the mapping from labor supply to the consumption aggregate:

In [7]:
m = BalancedBudget(nest, L = db('L2C').droplevel('nn').unique())
# m.initStuff(db = db)